In [1]:
"""
First insights:

* If an input is trained, it will stay stable, even if it receives less inputs in a second run
* If an input was trained less in a first run, it will strenghten its connections later if its stimuated more often then
* But in this case it will take much more time, since only few spikes are available which takes some time to build up
* If both inputs are trained strongly, further activating patterns have not really any impact anymore
-> An input aquires a specific amount of neurons and stably saturates
-> If it's relatively weak in the beginning it can still become saturates, but then it takes longer

Next steps:
* What happens when the network is bigger?
* What happens when the input it smaller/bigger?
* What happens when a third input comes into play?
* Add noise to allow re-allocation

"""
print('')

In [2]:
import os
import sys

path = '/home/seq/pelenet'
sys.path.append(path)
os.chdir(path)

In [3]:
# Allow reload of objects
%load_ext autoreload
%autoreload

from pelenet.utils import Utils
from pelenet.experiments.assemblies import AssemblyExperiment

# Official modules
import numpy as np
import nxsdk.api.n2a as nx
import matplotlib
import matplotlib.pyplot as plt
from time import time
from copy import deepcopy

In [4]:
# Overwrite default parameters (pelenet/parameters/ and pelenet/experiments/random.py)
parameters = {
    # Experiment
    'seed': 1,  # Random seed
    'trials': 20,  # Number of trials
    'stepsPerTrial': 50,  # Number of simulation steps for every trial
    'isReset': True,  # Activate reset after every trial
    # Network
    'reservoirExSize': 400,  # Number of excitatory neurons
    'reservoirConnPerNeuron': 35, #35 # Number of connections per neuron
    # Noise
    'isNoise': True,
    # Neurons
    'refractoryDelay': 2, # Refactory period
    'voltageTau': 100, #100 # Voltage time constant
    'currentTau': 6, #5 # Current time constant
    'thresholdMant': 1200, #1200  # Spiking threshold for membrane potential
    # Plasticity
    'isLearningRule': True,
    'learningEpoch': 4,
    'learningRule': '2^-2*x1*y0 - 2^-2*y1*x0 + 2^-4*x1*y1*x0*y0 - 2^-3*y0*w*w', #'2^-2*x1*y0 - 2^-2*y1*x0 + 2^-4*x1*y1*y0 - 2^-3*y0*w*w',
    # Input
    'inputIsVary': True,
    'inputVaryProbs': [0.1, 0.9],
    'inputGenSpikeProb': 0.8,  # Probability of spikes for the spike generators
    'inputNumTargetNeurons': 40,  # Number of neurons targeted by the spike generators
    'inputSteps': 30,  # Number of steps the input is active
    # Probes
    'isExSpikeProbe': True,  # Probe excitatory spikes
    'isInSpikeProbe': True,  # Probe inhibitory spikes
    'isWeightProbe': True  # Probe weight matrix at the end of the simulation
}

In [5]:
# Initilizes the experiment and utils
exp = AssemblyExperiment(name='assemblies', parameters=parameters)
utils = Utils.instance()

In [6]:
# Build network
exp.build()

# Get spectral radius
utils.getSpectralRadius(exp.net.initialWeights)

0.922

In [7]:
exp.net.numCoresUsed

25

In [8]:
# Run network
exp.run()

INFO:DRV:  Connecting to 127.0.0.1:46869
INFO:DRV:      Host server up..............Done 0.68s
INFO:DRV:      Encoding axons/synapses.....Done 0.41s
Compiling in directory : /home/seq/.local/share/virtualenvs/seq-r0_WvjRo/lib/python3.5/site-packages/nxsdk/driver/compilers/../../../temp/1606406847.1417167
Include dir is : /home/seq/.local/share/virtualenvs/seq-r0_WvjRo/lib/python3.5/site-packages/nxsdk/graph/nxinputgen/../../snips/ -I/home/seq/pelenet/pelenet/snips/
Compiling -c /home/seq/.local/share/virtualenvs/seq-r0_WvjRo/lib/python3.5/site-packages/nxsdk/driver/compilers/../../../temp/1606406847.1417167/initnetwork_0_0.c
gcc -Wall -I/home/seq/.local/share/virtualenvs/seq-r0_WvjRo/lib/python3.5/site-packages/nxsdk/graph/nxinputgen/../../snips/ -I/home/seq/pelenet/pelenet/snips/ -Inxsdk/include -fno-stack-protector -msoft-float -m32 -march=i586 -fno-builtin-printf -fno-builtin-sprintf -fno-builtin-fprintf -fno-builtin-memcpy -fstrict-volatile-bitfields -Os -DNDEBUG -std=c99 -o /home/

SystemExit: 1

In [ ]:
# Plot spike trains of the excitatory (red) and inhibitory (blue) neurons
exp.net.plot.reservoirSpikeTrain(figsize=(12,6))

In [ ]:
# Weight matrix before learning
exp.net.plot.initialExWeightMatrix()

In [ ]:
# Weight matrix after learning
exp.net.plot.trainedExWeightMatrix()

In [ ]:
# Sorted weight matrix after learning
expSupportMask = utils.getSupportWeightsMask(exp.net.trainedWeightsExex[-1])
exp.net.plot.weightsSortedBySupport(expSupportMask, exp.net.trainedWeightsExex[-1])

In [ ]:
inputs = utils.getInputLetterList(exp.net.inputTrials)

In [ ]:
supportMasks = utils.getSupportMasksForAllTrials(exp.net.initialWeights.exex, exp.net.trainedWeightsExex)

In [ ]:
exp.net.plot.supportShare(supportMasks)

In [ ]:
exp.net.plot.supportTurnover(supportMasks)

In [ ]:
# New parameters
newParameters = {
    **parameters,
    'trials': 50, #100,
    'inputVaryProbs': [0.9, 0.1]
}

In [ ]:
# Initilizes a second experiment
exp2 = AssemblyExperiment(name='assemblies-retrain', parameters=newParameters)

In [ ]:
#np.array_equal(exp.initialMasks.exex.toarray(), exp2.initialMasks.exex.toarray())

In [ ]:
# Build network
weights = deepcopy(exp.net.initialWeights)
weights.exex = exp.net.trainedWeightsExex[-1]
exp2.buildWithGivenMaskAndWeights(exp.net.initialMasks, weights)

In [ ]:
# Run network
exp2.run()

In [ ]:
# Plot spike trains of the excitatory (red) and inhibitory (blue) neurons
#exp2.net.plot.reservoirSpikeTrain(figsize=(12,6))

In [ ]:
# Weight matrix before learning
exp2.net.plot.initialExWeightMatrix()

In [ ]:
# Weight matrix after learning
exp2.net.plot.trainedExWeightMatrix()

In [ ]:
# Sorted weight matrix after learning
exp2SupportMask = utils.getSupportWeightsMask(exp2.net.trainedWeightsExex[-1])
exp2.net.plot.weightsSortedBySupport(exp2SupportMask, exp2.net.trainedWeightsExex[-1])

In [ ]:
supportMasks = utils.getSupportMasksForAllTrials(exp2.net.initialWeights.exex, exp2.net.trainedWeightsExex)

In [ ]:
exp2.net.plot.supportShare(supportMasks)
# TODO More trials do not help, need more noise?

In [ ]:
exp2.net.plot.supportTurnover(supportMasks)